In [88]:
# -------------------------------------------- Base FBA package --------------------------------------------
# -*- coding: utf-8 -*-

from __future__ import absolute_import

#import logging
from optlang.symbolics import Zero
from numpy import log as ln
import cobra
import re

bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)

#Adding a few exception classes to handle different types of errors
class FeasibilityError(Exception):
    """Error in FBA formulation
    """
    if Exception == '':
        pass
    
    if Exception == '':
        pass
    

#Base class for FBA packages
class BaseFBAPkg:
    def __init__(self, model, name, object, object_type = 'reaction', variable_types = {'concentration': 'float', 'lnconc': 'float'}, constraint_types = {'concentration': 'float'}, parent=None):
        '''Intantiate the model
        'model' (COBRA object): The COBRApy model object
        'name' (Python object, string): The name of the model
        'object' (COBRA object): The name of a COBRA reaction or metabolite, although, the former is the essential intention of the API  
        'object_type' (Python object, string): A description of the COBRA object, which is used to apply the pertinent code for the passed object
        'variable_types' (Python object, dictionary): The types and variables examples for the model variables
        'constraint_types' (Python object, dictionary): The names and values of the model constraints
        'parent' (Python object, boolean): The categorical description of the model 
        '''
        self.model = model
        self.name = name
        self.childpkgs = dict()
        self.parentpkg = parent
        self.constraints = dict()
        self.variables = dict()
        self.parameters = dict()
        self.variable_types = variable_types
        self.constraint_types = constraint_types
        for type in variable_types:
            self.variables[type] = dict()
        for type in constraint_types:
            self.constraints[type] = dict()
    

        from scipy.constants import calorie
        import pandas
        import re

        #import modelseedpy      
        #modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
        #modelseed = modelseedpy.biochem.from_local(modelseed_path)
        
        # import the Gibbs data from the TMFA supplementary excel file
        reactions_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Reactions')
        reactions_data.columns = reactions_data.iloc[0]
        reactions_data.drop(reactions_data.index[0], axis = 0, inplace = True)
        reactions_data.head()
        reactions_dict = {}
        for index, row in reactions_data.iterrows():
            reaction_abbreviation = reactions_data.at[index, 'iJR904 Abbreviation']
            reaction_name = reactions_data.at[index, 'Reaction name']
            reaction_gibbs = reactions_data.at[index, 'Estimated Gibbs free energy change of reaction (kcal/mol)']
            
            reactions_dict[reaction_abbreviation] = {'name': reaction_name, 'gibbs': reaction_gibbs}
        
        self.thermo_reactions = reactions_dict
        
        compounds_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Compounds')
        compounds_dict = {}
        for index, row in compounds_data.iterrows():
            compound_abbreviation = compounds_data.at[index, 'iJR904 Abbreviation']
            compound_name = compounds_data.at[index, 'Compound Name']
            compound_charge = compounds_data.at[index, 'Charge at pH 7*']
            try:
                compound_gibbs = float(compounds_data.at[index, 'Estimated Gibbs free energy of formation (kcal/mol)']) / calorie
            except:
                compound_gibbs = 0
            
            compounds_dict[compound_abbreviation] = {'name': compound_name, 'gibbs': compound_gibbs, 'charge': compound_charge}
        
        self.thermo_compounds = compounds_dict      
        
        # expand the metabolic thermodynamic data of the reaction in the argument 
        if object_type == 'reaction':
            reaction_metabolites = {}
            for metabolite in object.metabolites:
                tmfa_name = re.sub('(_.)', '', metabolite.id)
                self.thermo_compounds[tmfa_name]['stoichiometry'] = object.metabolites[metabolite]

            #for compound in self.thermo_compounds:
            #    print(compound)
        
        elif object_type == 'metabolite':
            tmfa_name = re.sub('(_.)', '', metabolite.id)
            self.thermo_compounds[tmfa_name] = object
        else:
            print('ERROR: The object_type is not compatible with this API.')
            
                        
    def validate_parameters(self, params, required, defaults):
        '''Validate the passed parameters with the required parameters and update the instance parameters with the default parameters
        'params' (Python object, dictionary): The parameters and values of the model
        'required' (Python object, list): The required parameters for the model 
        'defaults' (Python object, dictionary): The default parameters and values for the model 
        '''
        missing_parameters = set(required) - set(params)
        missing_string = ', '.join(list(missing_parameters))
        if len(missing_parameters) > 1:
            raise ValueError('The required parameters < {} > are missing!'.format(missing_string))
        elif len(missing_parameters) == 1:
            raise ValueError('The required parameter < {} > is missing!'.format(missing_string))
        
        self.parameters = params
        for key in defaults:
            if key not in params:
                self.parameters[key] = defaults[key]
        
        
    def build_variable(self, type, lower_bound, upper_bound, vartype, object, object_type = 'reaction'):
        '''Create variables of the specified type in the COBRA model
        'type' (Python object, string): The variable type within which variables will be created
        'lower_bound' (Python object, float): The lower bound value for the added variable
        'upper_bound' (Python object, float): The upper bound value for the added variable
        'vartype' (Python object, string): The variable type as either 'continuous', 'integer', or 'binary' 
        'object' (COBRA object): The COBRA entity into which a variable will be build
        'object_type' (Python object): The variable type of the COBRA object 
        '''
        # assign a variable name based upon the passed arguments
        if object_type == "none":
            count = len(self.variables[type])
            name = str(count + 1)
        elif object_type == "string":
            name = object
        elif object_type in ['reaction', 'metabolite']:
            self.variable_types[type] = vartype  
            name = object.id
            
        # add an optlang variable, when the variable is undefined
        #raise ValueError('The object name {} is not recognized by your model'.format(missing_string))
        if name in self.variables[type]:
            print('ERROR: The {} constraint already exists with a value of {}.'.format(name, self.variables[type][name]))
            self.variables[name] = [lower_bound, upper_bound]
            variable_definition = self.variables[name] 
        
        elif name not in self.variables[type]:
            variable_name = name + "_" + type
            variable_definition = self.model.problem.Variable(name = variable_name, lb = lower_bound, ub = upper_bound, type = vartype)
            self.model.add_cons_vars(variable_definition)
            self.variables[name] = [lower_bound, upper_bound]
            
        return variable_definition
        
        
    def build_constraint(self, constraint_expression, type, lower_bound, upper_bound, object, object_type = 'reaction', coef = {}):
        '''Create constraints for the COBRA model
        'type' (Python object, string): The type of the constraint that will be created 
        'lower_bound' (Python object, float): The lower bound value for the added constraint
        'upper_bound' (Python object, float): The upper bound value for the added constraint
        'object_type' (Python object, string): The variable type of the COBRA object 
        'object' (Python object, string): The variable name when the name is defined
        'coef' (Python object, dictionary): The set of coefficients that define the COBRA model
        '''
        # assign a constraint name based upon the passed arguments
        if object_type == "none":
            count = len(self.constraints[type])
            name = str(count + 1)
        elif object_type == "string":
            name = object
        elif object_type in ['reaction', 'metabolite']:
            self.constraint_types[type] = coef
            name = object.id
                   
        # add an optlang constraint, when the constraint is undefined 
        if name in self.constraints[type]:
            print('ERROR: The {} constraint already exists with a value of {}.'.format(name, constraints[type][name]))
        elif name not in self.constraints[type]:
            constraint_name = '{}_{}'.format(name, type)
            self.constraints[type][constraint_name] = self.model.problem.Constraint(expression = constraint_expression, lb = lower_bound, ub = upper_bound, name = constraint_name)
            self.model.add_cons_vars(self.constraints[type][constraint_name])
            self.model.solver.update()
            if len(coef) > 0:
                self.constraints[type][constraint_name].set_linear_coefficients(coef)
                
            self.model.solver.update()
            self.constraints[type][constraint_name] = [lower_bound, upper_bound]
        
        return self.constraints[type][constraint_name]
    
    
    def all_variables(self):
        '''Quantify the variables in the class
        '''
        vars = {}
        for child in self.childpkgs:
            for type in child.variables:
                vars[type] = child.variables[type]

        for type in self.variables:
            vars[type] = self.variables[type]
        
        return vars
    
    
    def all_constraints(self):
        '''Quantify the constraints in the class
        '''
        consts = {}
        for child in self.childpkgs:
            for type in child.constraints:
                consts[type] = child.constraints[type]

        for type in self.constraints:
            consts[type] = self.constraints[type]
            
        return consts
    
    
    def revert_to_original(self):
        '''Remove all variables from the model instance and removes added constraints to the COBRA model
        '''               
        global model
        # remove added variables and constants from the model by re-uploading the COBRA model  
        model = cobra.io.load_json_model(bigg_model_path)
    
        # completely clear the model instance
        self.variables = dict()
        self.constraints = dict()
        
        
# ---------------------------------------------- Revbin -------------------------------------------------

# The base class for FBA packages is inherited
class RevBinPkg(BaseFBAPkg):
    def __init__(self, model, object):
        '''Redefining the inherited __init__ function
        'model' (COBRA object): The COBRApy FBA model
        '''
        BaseFBAPkg.__init__(self, model = model, object = object, object_type = 'reaction', name = "reversible binary", variable_types = {"revbin":"reaction", 'forv':'reaction', 'revv':'reaction'}, constraint_types = {"revbinF":"reaction", "revbinR":"reaction"})

        if object.id not in self.variables['revbin']:
            self.variables['revbin'][object.id] = object.forward_variable # should be forv

    def build_constraint(self, object):
        '''Build constraints through the inherited function and the calculated coefficient fluxes
        'object' (Python object, string): The variable name when the name is defined
        '''
        # define the variables that are used in the constraints
        BaseFBAPkg.build_variable(self, type = "revbin", lower_bound = 0, upper_bound = 1, vartype = "binary", object = object)
        
        BaseFBAPkg.build_variable(self, type = "forv", lower_bound = 0, upper_bound = 1000, vartype = "continuous", object = object)
        
        BaseFBAPkg.build_variable(self, type = "revv", lower_bound = 0, upper_bound = 1000, vartype = "continuous", object = object)
        
        # define the constraints of the system
        coef = {self.variables["revbin"][object.id]:-1000, object.forward_variable:1}
        built_forward_constraint = BaseFBAPkg.build_constraint(self, constraint_expression = Zero, type = "revbinF", lower_bound = None, upper_bound = 0, coef = coef, object = object)
        
        coef = {self.variables["revbin"][object.id]:1000, object.reverse_variable:1}
        built_backward_constraint = BaseFBAPkg.build_constraint(self, constraint_expression = Zero, type = "revbinR", lower_bound = None, upper_bound = 1000, coef = coef, object = object)
        
        return built_backward_constraint
    
    
    def build_package(self, filter = model.reactions):
        '''Build variables and constraints through the inherited function
        'filter' (Python object, list): The accepted list of reactions that will be built into the model
        '''
        preexisting_reactions = []
        for reaction in self.model.reactions:
            # Unfiltered reactions are constructed through the aforementioned functions
            if reaction.id not in filter:
                self.build_constraint(reaction)
            elif reaction.id in filter:
                preexisting_reactions.append(reaction.id)
                
        reactions = ', '.join(preexisting_reactions)
        print('ERROR: The reactions {} already exist in the model.'.format(reactions))
                
                
# ------------------------------------------ Simple Thermo package ------------------------------------------                

# The base class for FBA packages is inherited
class SimpleThermoPkg(BaseFBAPkg):
    def __init__(self, model, object):
        '''Redefining the inherited __init__ function
        'model' (COBRA object): The COBRApy FBA model
        '''        
        from scipy.constants import physical_constants, kilo, R
        from numpy import log as ln        

        F = physical_constants['Faraday constant'][0]
                
        # execute the parent __init__ and arbitrarily assign potentials to each metabolite 
        BaseFBAPkg.__init__(self, model = model, name = "simple thermo", object = object, object_type = 'reaction', variable_types = {"potential":"metabolite", 'concentration_potential': 'metabolite'}, constraint_types = {"thermo":"reaction"})
        self.childpkgs["reversible binary"] = RevBinPkg(model, object = model.reactions.get_by_id('PFK'))
        
        # parameterize the initial chemical concentrations 
        self.validate_parameters(params = self.parameters, required = [], defaults = {
            "default_conc_range": [0.001, 20],  # an arbitrary range
            "custom_concentration_constraints": {"glc-D": [10,10],  #cpd00027,  
                                                'co2': [20, 24], #cpd00011, as bicarbonate, E. B. Brown and Richard L. Clancy, 1964
                                                'h': [0.0053, 0.0053], #cpd00067, Battaglia, Hellegers, & Seeds, 1965
                                                'o2': [0.672, 0.672] #cpd00007, 0.3 mL / dL serum O2 concentration
                                                 }
        })
        
        # calculate the total energy of a reaction based upon the values for each constitutent metabolite in the reaction 
        delta_g = 0
        sum_concentration_potential = 0
        sum_electro_potential = 0
        temperature = 25 # degrees kelvin  
        self.variables['concentration_potential'] = {}
        self.variables['electro_potential'] = {}
        self.parameters['activity_coefficient'] = {}
        self.variables['total_energy'] = {}
        for metabolite in object.metabolites:
            tmfa_name = re.sub('(_.)', '', metabolite.id)
            
            # calculate the concentration range for the chemical specie
            if tmfa_name not in self.parameters['custom_concentration_constraints']:
                concentration_range = self.parameters['default_conc_range']
                self.thermo_compounds[tmfa_name]['concentration'] = sum(concentration_range) / len(concentration_range)
            elif tmfa_name in self.parameters['custom_concentration_constraints']:
                concentration_range = self.parameters['custom_concentration_constraints'][tmfa_name]
                self.thermo_compounds[tmfa_name]['concentration'] = sum(concentration_range) / len(concentration_range)
                
                
            # calculate the electrochemical potential term
            if metabolite.compartment == 'c':
                ph_gradient = 2 # arbitrary value
            elif metabolite.compartment == 'e':
                ph_gradient = 0 # by defintion of a zero ph gradient between the metabolite compartment and the extracellular compartment
                
            psi_electro_potential = 33.33 * ph_gradient - 143.33  # millivolts, equation 9 from the TMFA paper 
            self.variables['electro_potential'][metabolite.id] = psi_electro_potential * F * self.thermo_compounds[tmfa_name]['charge'] * kilo

            
            # calculate the concentration potential term from the average concentration
            if metabolite.id not in self.parameters['activity_coefficient']:
                activity_coefficient = 1
            elif metabolite.id in self.parameters['activity_coefficient']:
                activity_coefficient = self.parameters['activity_coefficient'][metabolite.id]
            
            self.variables['concentration_potential'][metabolite.id] = self.thermo_compounds[tmfa_name]['stoichiometry'] * ln(self.thermo_compounds[tmfa_name]['concentration'] * activity_coefficient)
            
            
            # sum the all energetic descriptions of each metabolite in a reaction
            sum_concentration_potential += self.variables['concentration_potential'][metabolite.id]
            sum_electro_potential += self.variables['electro_potential'][metabolite.id]
            delta_g += self.thermo_compounds[tmfa_name]['gibbs']                
            

        # calculation of the total energetic potential of the reaction based upon the metabolite calculations
        self.variables['total_energy'][object.id] = delta_g + R * temperature * sum_concentration_potential + sum_electro_potential  
                                                
    def build_constraint(self, object):
        '''Build constraints through the inherited function and the calculated variable coeffiients 
        'object' (Python object, string): The variable name when the name is defined
        '''
        delta_g = 0 #sum(st(i,j)*p(j))
        
        for metabolite in object.metabolites:
            tmfa_name = re.sub('(_.)', '', metabolite.id)
            delta_g += self.thermo_compounds[tmfa_name]['stoichiometry'] * (self.thermo_compounds[tmfa_name]['gibbs'] + self.variables['concentration_potential'][metabolite.id] + self.variables['electro_potential'][metabolite.id])
            
        if object.id not in self.childpkgs["reversible binary"].variables['revbin']:
            self.childpkgs["reversible binary"].variables['revbin'][object.id] = object.forward_variable
            
        if object.reversibility:
            binary = 0
        elif not object.reversibility:
            binary = 1
        else:
            print('ERROR: The reaction object possesses unpredictable data structure.')

        # create thermodynamic constraints and the associated variables
        for metabolite in self.model.metabolites:
            BaseFBAPkg.build_variable(self, type = "potential", lower_bound = 0, upper_bound = 1000, vartype = "continuous", object = metabolite)
            
        coef = {self.childpkgs["reversible binary"].variables["revbin"][object.id] : 1000}
            
        return BaseFBAPkg.build_constraint(self, constraint_expression = Zero, type = "thermo", lower_bound = 0, upper_bound = 1000, coef = coef, object = object)

        
    def build_package(self, filter = model.reactions):
        '''Build variables and constraints through the inherited function
        'filter' (Python object, list): The accepted list of reactions that will be built into the model
        '''
        self.childpkgs["reversible binary"].build_package(filter)
                
                
# ------------------------------------------ Full Thermo package ------------------------------------------



# The base class for FBA packages is inherited
class FullThermoPkg(BaseFBAPkg):
    def __init__(self, model, object):
        '''Redefining the inherited __init__ function and importing thermodynamic data
        'model' (COBRA object): The COBRApy FBA model
        '''                 
        # execute the base __init__ file
        BaseFBAPkg.__init__(self, model = model, object = object, name = "full thermo", variable_types = {"lnconc":"metabolite", 'thermo': 'reactions'}, constraint_types = {"potentialc":"metabolite", 'thermo': 'metabolite'})
        self.childpkgs["simple thermo"] = SimpleThermoPkg(model, object = model.reactions.get_by_id('PFK'))
        self.childpkgs["potential"] = RevBinPkg(model, object = model.reactions.get_by_id('PFK'))
    
        
    '''def build_variable(self, object): #, modelseed = modelseed):   
        #flux variability analysis? 
        
        from numpy import log as ln
        if object.id in self.parameters["custom_concentration_constraints"]:
            lb = ln(self.parameters["custom_concentration_constraints"][object.id][0])
            ub = ln(self.parameters["custom_concentration_constraints"][object.id][1])
        else:
            lb = ln(self.parameters["default_min_conc"])
            ub = ln(self.parameters["default_max_conc"])
            
        return BaseFBAPkg.build_variable(self, type = "lnconc", lower_bound = lb, upper_bound = ub, vartype = "continuous", object = object)'''
    
    
    def build_constraint(self, model, object, type = 'thermo'): #, modelseed = modelseed):
        '''Build constraints through the inherited function and the calculated variable coeffiients 
        'object' (Python object, string): The variable name when the name is defined
        Notes - Equation 14 in the TMFA paper, with the addition of the (charge * compartment_potential) term?
        '''
            
        # calculate the parameters for the constraint expression calculation               
        constant = 20  # arbitrary value
        coef = {self.childpkgs["potential"].variables["revbin"][object.id] : 1000}
        constraint_name = '{}_{}'.format(object.id, type)
        
        if constraint_name not in model.constraints:
            built_constraint = BaseFBAPkg.build_constraint(self, constraint_expression = constraint_expression, type = type, lower_bound = constant, upper_bound = constant, coef = coef, object = object)
            
        else:
            print('ERROR: The {} contraint already exists in the model: {}, ub: {}, lb: {}'.format(constraint_name, model.constraints[constraint_name], object.upper_bound, object.lower_bound))
            object.upper_bound = constant
            object.lower_bound = constant
            #model.constraints[constraint_name] = constraint_expression
            print('The {} contraint is updated in the model: {}, ub: {}, lb: {}'.format(constraint_name, model.constraints[constraint_name], object.upper_bound, object.lower_bound))
            built_constraint = None
            
        return built_constraint

        
    def build_package(self, filter = model.reactions):
        '''Create the final model package
        'filter' (Python object, list): The accepted list of reactions that will be built into the model
        Notes - The concentrations are expressed in millimolar
        '''                   
        self.childpkgs["simple thermo"].build_package(filter)
        
        # The concentration variable and potential constraint are built
        for metabolite in self.model.metabolites:
            BaseFBAPkg.build_variable(self, type = "thermo", lower_bound = 0, upper_bound = 1000, vartype = "continuous", object = metabolite)
            
        for reaction in self.model.reactions:
            # Unfiltered reactions are constructed through the aforementioned functions
            if reaction.id not in filter:
                self.build_constraint(reaction)


#------------------------------------------------ Test ---------------------------------------------------

# Base FBA Package
announcement = 'BaseFBApkg'
print(announcement, '\n', '='*len(announcement))


# __init__ 
print('\n+ __init__')
base = BaseFBAPkg(model = model, name = 'test_model', object = model.reactions.get_by_id('PFK'), object_type = 'reaction')      
print('model used: ', base.model)
print('instance name: ', base.name)
print('variable types: ', base.variable_types)
print('constraint types: ', base.constraint_types)


# validate_parameters 
print('\n+ validate_parameters')
params = {'a':2, 'b':4, 'c':3}
required = ['a', 'b', 'c']
defaults = {'a':1, 'b':1, 'c':1, 'd':1}

print('parameters before: ', base.parameters)
base.validate_parameters(params, required, defaults)
print('parameters after: ', base.parameters)


# build_variable
print('\n+ build_variable')
print('variables before: ', base.variables)
base.build_variable(type = 'concentration', lower_bound = 10, upper_bound = 133, vartype = 'continuous', object = model.reactions.get_by_id('PFK'))
print('variables after: ', base.variables)


# build_constraint
print('\n+ build_constraint')
print('constraints before: ', base.constraints)
base.build_constraint(constraint_expression = 0, type = 'concentration', lower_bound = 10, upper_bound = 133, object = model.reactions.get_by_id('PFK'))
print('constraints after: ', base.constraints)


# all_variables
print('\n+ all_variables')
print('model variables: ', base.all_variables())


# all_constraints
print('\n+ all_constraints')
print('model constraints: ', base.all_constraints())


# clear
print('\n+ revert_to_original')
print('instance variables before: ', base.variables)
print('instance constraints before: ', base.constraints)
model2 = cobra.io.load_json_model(bigg_model_path)

variables = []
constraints = []
for var in model.variables:
    variables.append(str(var))
for constraint in model.constraints:
    constraints.append(str(constraint))
    
variables2 = []
constraints2 = []
for var in model2.variables:
    variables2.append(str(var))
for constraint in model2.constraints:
    constraints2.append(str(constraint))
    
difference_variables = set(variables) - set(variables2)
print('added variables before: ', difference_variables)
difference_constraints = set(constraints) - set(constraints2)
print('added constraints before: ', difference_constraints)

base.revert_to_original()

print('\ninstance variables after: ', base.variables)
print('instance constraints after: ', base.constraints)

variables = []
constraints = []
for var in model.variables:
    variables.append(str(var))
for constraint in model.constraints:
    constraints.append(str(constraint))
    
variables2 = []
constraints2 = []
for var in model2.variables:
    variables2.append(str(var))
for constraint in model2.constraints:
    constraints2.append(str(constraint))
    
difference_variables = set(variables) - set(variables2)
print('added variables after: ', difference_variables)
difference_constraints = set(constraints) - set(constraints2)
print('added constraints after: ', difference_constraints)



# Revbin
announcement = 'Revbin'
print('\n', announcement, '\n', '='*len(announcement))


# __init__
print('\n+ __init__')
revbin = RevBinPkg(model, object = model.reactions.get_by_id('PFK'))
print('model used: ', revbin.model)
print('instance name: ', revbin.name)
print('variable types: ', revbin.variable_types)
print('constraint types: ', revbin.constraint_types)


# build_constraint
print('\n+ build_constraint')
print('constraints before: ', revbin.constraints)
revbin.build_constraint(object = model.reactions.get_by_id('PFK'))
print('constraints after: ', revbin.constraints)


# build_package
print('\n+ build_package')
print('constraints before: ', revbin.constraints)
revbin.build_package()
print('constraints after: ', revbin.constraints)


# Simple Thermo
announcement = 'Simple Thermo'
print('\n', announcement, '\n', '='*len(announcement))


# __init__
print('\n+ __init__')
simple_thermo = SimpleThermoPkg(model, object = model.reactions.get_by_id('PFK'))
print('model used: ', simple_thermo.model)
print('instance name: ', simple_thermo.name)
print('variable types: ', simple_thermo.variable_types)
print('constraint types: ', simple_thermo.constraint_types)


# build_constraint
print('\n+ build_constraint')
print('constraints before: ', simple_thermo.constraints)
simple_thermo.build_constraint(object = model.reactions.get_by_id('PFK'))
print('constraints after: ', simple_thermo.constraints)


# build_package
print('\n+ build_package')
print('constraints before: ', simple_thermo.constraints)
simple_thermo.build_package()
print('constraints after: ', simple_thermo.constraints)


# Full Thermo
announcement = 'Full Thermo'
print('\n', announcement, '\n', '='*len(announcement))


# __init__
print('\n+ __init__')
full_thermo = FullThermoPkg(model, object = model.reactions.get_by_id('PFK'))
print('model used: ', full_thermo.model)
print('instance name: ', full_thermo.name)
print('variable types: ', full_thermo.variable_types)
print('constraint types: ', full_thermo.constraint_types)


# build_variable
'''print('\n+ build_variable')
print('variables before: ', full_thermo.variables)
full_thermo.build_variable(object = model.reactions.get_by_id('PFK'))
print('variables after: ', full_thermo.variables)'''


# build_constraint
print('\n+ build_constraint')
print('constraints before: ', full_thermo.constraints)
full_thermo.build_constraint(model = model, object = model.reactions.get_by_id('PFK'))
print('constraints after: ', full_thermo.constraints)


# build_package
print('\n+ build_package')
print('constraints before: ', full_thermo.constraints)
full_thermo.build_package()
print('constraints after: ', full_thermo.constraints)


# printing the test LP file
with open('test.lp', 'w') as output:
    output.write(str(model.solver))

BaseFBApkg 

+ __init__
model used:  e_coli_core
instance name:  test_model
variable types:  {'concentration': 'float', 'lnconc': 'float'}
constraint types:  {'concentration': 'float'}

+ validate_parameters
parameters before:  {}
parameters after:  {'a': 2, 'b': 4, 'c': 3, 'd': 1}

+ build_variable
variables before:  {'concentration': {}, 'lnconc': {}}
variables after:  {'concentration': {}, 'lnconc': {}, 'PFK': [10, 133]}

+ build_constraint
constraints before:  {'concentration': {}}
constraints after:  {'concentration': {'PFK_concentration': [10, 133]}}

+ all_variables
model variables:  {'concentration': {}, 'lnconc': {}, 'PFK': [10, 133]}

+ all_constraints
model constraints:  {'concentration': {'PFK_concentration': [10, 133]}}

+ revert_to_original
instance variables before:  {'concentration': {}, 'lnconc': {}, 'PFK': [10, 133]}
instance constraints before:  {'concentration': {'PFK_concentration': [10, 133]}}
added variables before:  {'10 <= PFK_concentration <= 133'}
added const

# Brainstorming

In [2]:
dictionary = {'a':1, 'b':2}
require = {'a':2, 'c':2}
set_1 = set(require) - set(dictionary)
print(set_1)

{'c'}


In [5]:
defaults = {"custom_concentration_constraints": {"cpd000027_e0": ["10","10"],
                                                'cpd000033_c0': ['3', '4']}}
print(defaults["custom_concentration_constraints"]['cpd000033_c0'][0])

3


In [16]:
import pandas
compound_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Compounds')
gibbs_compounds = compound_data['Estimated Gibbs free energy of formation (kcal/mol)']
# = thermodynamic_data['Compounds']
display(gibbs_compound_data)
#display(thermodynamic_data)

0     -152.39300
1     -181.65000
2      -79.54030
3     -587.09200
4       -7.09536
         ...    
623   -181.77700
624   -182.11500
625      Unknown
626          NaN
627          NaN
Name: Estimated Gibbs free energy of formation (kcal/mol), Length: 628, dtype: object

In [59]:
import cobra
import modelseedpy

reaction = 'PFK'

bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)
reaction = model.reactions.get_by_id(reaction)



'''print('cobra reaction: ', reaction)
print('cobra reaction name: ', reaction.name)

for metabolite in reaction.metabolites:
    print('cobra reaction metabolites: ', metabolite.id)
    print('cobra reaction metabolites: ', metabolite.id)'''

'''modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
modelseed = modelseedpy.biochem.from_local(modelseed_path)
modelseed_reaction = modelseed.get_name(reaction_name)
print('modelseed reaction: ', modelseed_reaction)'''

dir(model)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_compartments',
 '_contexts',
 '_id',
 '_populate_solver',
 '_repr_html_',
 '_set_id_with_model',
 '_solver',
 '_tolerance',
 '_trimmed',
 '_trimmed_genes',
 '_trimmed_reactions',
 'add_boundary',
 'add_cons_vars',
 'add_groups',
 'add_metabolites',
 'add_reaction',
 'add_reactions',
 'annotation',
 'boundary',
 'compartments',
 'constraints',
 'copy',
 'demands',
 'description',
 'exchanges',
 'genes',
 'get_associated_groups',
 'get_metabolite_compartments',
 'groups',
 'id',
 'medium',
 'merge',
 'metabolites',
 'name

In [22]:
import modelseedpy

modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
modelseed = modelseedpy.biochem.from_local(modelseed_path)

#dir(modelseed.get_seed_compound('cpd00067'))
#dir(modelseed)
print(modelseed.get_seed_compound('cpd00067').data['abbreviation'])
print(modelseed.get_seed_compound('cpd00011').data['abbreviation'])
print(modelseed.get_seed_compound('cpd00027').data['abbreviation'])
#modelseed.get_seed_compound('cpd00067').aliases


h
co2
glc-D


In [92]:
import pandas

reactions_data = pandas.read_excel('Supplementary file.xls', sheet_name = 'Reactions')
reactions_data.columns = reactions_data.iloc[0]
reactions_data.drop(reactions_data.index[0], axis = 0, inplace = True)
reactions_data.head()

,iJR904 Abbreviation,Reaction name,Estimated Gibbs free energy change of reaction (kcal/mol),Standard uncertainty in estimate (kcal/mol)
1,ALAR,alanine racemase,0,0.50000
2,ASNN,L-asparaginase,-4.20409,1.16973
3,ASNS2,asparagine synthetase,-2.54684,1.40088
4,ASNS1,asparagine synthase (glutamine-hydrolysing),-6.75092,1.04603
5,ASPT,L-aspartase,2.40657,1.02896


In [82]:
import cobra

reaction_id = 'PFK'

bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)
'''for reaction in model.reactions:
    print(reaction)'''

reaction = model.reactions.get_by_id(reaction_id)
print(reaction.reaction)
print(reaction.metabolites)
print('initial :', reaction.forward_variable)
#reaction.forward_variable = 24 * 'PDK'
#print('final :', reaction.forward_variable)
print(reaction.reverse_variable)
#print(reaction.constraints)

for metabolite in reaction.metabolites:
    print(metabolite.id)
    print(reaction.metabolites[metabolite])
    print(metabolite.compartment)

#print(model.constraints[reaction.id])
print(type(model.solver))

#dir(model.constraints)
#for constraint in model.constraints:
#    print(constraint)

atp_c + f6p_c --> adp_c + fdp_c + h_c
{<Metabolite adp_c at 0x2a9a818bec8>: 1.0, <Metabolite atp_c at 0x2a9a818bfc8>: -1.0, <Metabolite f6p_c at 0x2a9a8197248>: -1.0, <Metabolite fdp_c at 0x2a9a8197288>: 1.0, <Metabolite h_c at 0x2a9a818b488>: 1.0}
initial : 0 <= PFK <= 1000.0
0 <= PFK_reverse_d24a6 <= -0.0
adp_c
1.0
c
atp_c
-1.0
c
f6p_c
-1.0
c
fdp_c
1.0
c
h_c
1.0
c
<class 'optlang.glpk_interface.Model'>


In [84]:
x = 4
y = x
x += 3

print(x)
print(y)

7
4
